# Part 3: Non-parametric Bayesian tests
<sup>http://ipg.idsia.ch/tutorials/2016/bayesian-tests-ml/ </sup>

<br>

### **Alessio Benavoli**, IDSIA
##### http://people.idsia.ch/~alessio/

<br>
<br>



### Nonparametric analysis

T-test, correlated t-test are part of  parametric statistics
<br>
<center><img src="plots/canvas.png" width="431"></center> 

In ML we use these **nonparametric** tests for
1. comparing classifiers
2. many other purposes (e.g., features, graph of a BN)



# What is nonparametric statistics?

# Frequentist definition

>*A nonparametric procedure is a **statistical procedure** that has a certain
**desirable properties** that hold under relatively **mild assumptions** regarding
the **underlying populations** from which the data are obtained.*


 
---
<sub><sub>Hollander, Myles, Douglas A. Wolfe, and Eric Chicken. Nonparametric statistical methods. John Wiley and Sons, 2013.</sub></sub>


## Confused
<center><img src="plots/confused.png" width="400px"></center>




# Frequentist definition

>*The term nonparametric is imprecise. The related term **distribution-free**
has a more precise meaning...The distribution-free property **enables one** to obtain the **distribution of the statistic** under the **null hypothesis** without specifying the underlying distribution of the data*

___
<sub><sub>Hollander, Myles, Douglas A. Wolfe, and Eric Chicken. Nonparametric statistical methods. John Wiley and Sons, 2013.</sub></sub>

## comparing classifiers:
* $m$ runs of the $k$-fold cross-validation accuracy for each dataset for **nbc** and for **aode**

<center><img src="plots/table0.png" width="751"></center> 


## How do we do that with NHST?

There is **no direct NHST** 

* that takes **that table** as input and
* returns as output a *statistical decision* about which classifier is better in all the datasets.

## NHST procedure 1/2
The **usual way** to address this problem with ** NHST** is:
    
1. compute the *mean difference* of accuracy for each dataset (averaging the differences
of accuracies obtained in the $m$ runs of the $k$-fold cross-validation);
   

## Data

<center><img src="plots/table0.png" width="851"></center> 

### NHST procedure 1/2

<center><img src="plots/table.png" width="951"></center>

## NHST procedure 2/2
     
2) perform NHST to **establish** if the two classifiers have **different performance or not**
based on these **mean differences** of accuracy.

Suggested (**Nonparametric**) Tests:
* **Wilcoxon signed-rank test** or **Sign test**

Why?
1. no assumption of  normality (distribution-free);
2. no assumption of commensurability across datasets; 
3. robust to outliers.
<br>
<sub><sub> J. Demsar. Statistical comparisons of classifiers over multiple data sets.  *Journal of Machine learning research* 2006 <sub><sub>

In [1]:
using Distributions, Gadfly, Compose, DataFrames, Fontconfig,  Cairo
include("/home/benavoli/Data/Work_Julia/Julia/Plots/plot_data1.jl")
include("/home/benavoli/Data/Work_Julia/Tutorial/Julia/Tests/Bsigntest.jl")
ClassID = readdlm("/home/benavoli/Data/Work_Julia/Tutorial/Julia/Data/ClassifierID.dat", ',')
ClassNames = readdlm("/home/benavoli/Data/Work_Julia/Tutorial/Julia/Data/ClassifierNames.dat", ',')
DatasetID = readdlm("/home/benavoli/Data/Work_Julia/Tutorial/Julia/Data/DatasetID.dat", ',');
DatasetNames = readdlm("/home/benavoli/Data/Work_Julia/Tutorial/Julia/Data/DatasetNames.dat", ',');
Percent_correct = readdlm("/home/benavoli/Data/Work_Julia/Tutorial/Julia/Data/Percent_correct.dat", ',');

cl1a=1 #nbc
cl2a=2 #aode
println("Comparison of ", ClassNames[cl1a,1], " vs. ", ClassNames[cl2a,1])
println()

#Compute mean accuracy
indi=find(x->x==cl1a,ClassID)
indj=find(x->x==cl2a,ClassID)
accNbc=Float64[]
accAode=Float64[]
for d=1:Int32(maximum(DatasetID))
    indd=find(x->x==d,DatasetID)
    indid=intersect(indi,indd)
    indjd=intersect(indj,indd)
    push!(accNbc,mean(Percent_correct[indid])/100)
    push!(accAode,mean(Percent_correct[indjd])/100)
end

Comparison of nbc vs. aode



    +(AbstractArray{T<:Any, 2}, WoodburyMatrices.SymWoodbury) at /home/benavoli/.julia/v0.4/WoodburyMatrices/src/SymWoodburyMatrices.jl:106
is ambiguous with: 
    +(DataArrays.DataArray, AbstractArray) at /home/benavoli/.julia/v0.4/DataArrays/src/operators.jl:276.
To fix, define 
    +(DataArrays.DataArray{T<:Any, 2}, WoodburyMatrices.SymWoodbury)
before the new definition.
    +(AbstractArray{T<:Any, 2}, WoodburyMatrices.SymWoodbury) at /home/benavoli/.julia/v0.4/WoodburyMatrices/src/SymWoodburyMatrices.jl:106
is ambiguous with: 
    +(DataArrays.AbstractDataArray, AbstractArray) at /home/benavoli/.julia/v0.4/DataArrays/src/operators.jl:300.
To fix, define 
    +(DataArrays.AbstractDataArray{T<:Any, 2}, WoodburyMatrices.SymWoodbury)
before the new definition.


## Wilcoxon signed-rank test
Typically used as follows.
* null hypothesis is that the **median** of the distribution is 0; 
* when the test rejects  the **null**, it claims that it is **significantly different from 0**.

Assumptions
* $z_1,\dots,z_q$ i.i.d. and generated from a *symmetric distribution*
* The test is miscalibrated if the distribution is not symmetric.



 ## Wilcoxon signed-rank test
 
 The test statistic is:
 $$
  \begin{array}{rcl}
  t=& \sum\limits_{1\leq i \leq j \leq q} t^+_{ij},
  \end{array}
$$

  where 

$$
  t^+_{ij}=\left\{\begin{array}{ll}
  1 & \textit{if } z_i \geq -z_j,\\
  0 & \textit{otherwise. } \\
  \end{array}\right.
  $$

Example: consider the two cases ${z}=\{-2,-1,4,5\}$ or ${z}=\{-1,4,5\}$, statistic is $t=7$
  and, respectively, $t=5$. 
  
$p$-value of the statistic is computed against the **null** (zero median).

## NBC - AODE

In [8]:
using HypothesisTests
p=plot_data1(cl1a,cl2a,"all datasets",accNbc-accAode,-0.15,0.1)
display(p) 
p1=pvalue(SignedRankTest(accNbc,accAode))
p2=pvalue(SignTest(accNbc,accAode))
@printf "p-value SignedRank=%0.8f    " p1 
@printf "p-value Sign=%0.8f\n" p2

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 DeltaAcc 
 
 
 -0.6 
 -0.5 
 -0.4 
 -0.3 
 -0.2 
 -0.1 
 0.0 
 0.1 
 0.2 
 0.3 
 0.4 
 0.5 
 -0.40 
 -0.39 
 -0.38 
 -0.37 
 -0.36 
 -0.35 
 -0.34 
 -0.33 
 -0.32 
 -0.31 
 -0.30 
 -0.29 
 -0.28 
 -0.27 
 -0.26 
 -0.25 
 -0.24 
 -0.23 
 -0.22 
 -0.21 
 -0.20 
 -0.19 
 -0.18 
 -0.17 
 -0.16 
 -0.15 
 -0.14 
 -0.13 
 -0.12 
 -0.11 
 -0.10 
 -0.09 
 -0.08 
 -0.07 
 -0.06 
 -0.05 
 -0.04 
 -0.03 
 -0.02 
 -0.01 
 0.00 
 0.01 
 0.02 
 0.03 
 0.04 
 0.05 
 0.06 
 0.07 
 0.08 
 0.09 
 0.10 
 0.11 
 0.12 
 0.13 
 0.14 
 0.15 
 0.16 
 0.17 
 0.18 
 0.19 
 0.20 
 0.21 
 0.22 
 0.23 
 0.24 
 0.25 
 0.26 
 0.27 
 0.28 
 0.29 
 0.30 
 0.31 
 0.32 
 0.33 
 0.34 
 0.35 
 -0.4 
 -0.2 
 0.0 
 0.2 
 0.4 
 -0.40 
 -0.38 
 -0.36 
 -0.34 
 -0.32 
 -0.30 
 -0.28 
 -0.26 
 -0.24 
 -0.22 
 -0.20 
 -0.18 
 -0.16 
 -0.14 
 -0.12 
 -0.10 
 -0.08 
 -0.06 
 -0.04 
 -0.02 
 0.00 
 0.02 
 0.04 
 0.06 
 0.08 
 0.10 
 0.12 
 0.14 
 0.16 
 0.18 
 0.20 
 0.22 
 0.24 
 0.26 
 0.28 
 0.30 
 0.32 
 0.34 
 0.36 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path fill="none" d="M-9.46,78.3 L -9.02 78.3 -8.58 78.3 -8.14 78.29 -7.7 78.29 -7.26 78.29 -6.82 78.29 -6.38 78.28 -5.94 78.27 -5.5 78.26 -5.06 78.25 -4.62 78.23 -4.19 78.21 -3.75 78.18 -3.31 78.14 -2.87 78.09 -2.43 78.03 -1.99 77.95 -1.55 77.86 -1.11 77.76 -0.67 77.64 -0.23 77.51 0.21 77.36 0.65 77.21 1.09 77.04 1.53 76.87 1.97 76.71 2.4 76.54 2.84 76.39 3.28 76.26 3.72 76.15 4.16 76.06 4.6 76.01 5.04 75.98 5.48 76 5.92 76.04 6.36 76.11 6.8 76.22 7.24 76.34 7.68 76.49 8.12 76.65 8.56 76.81 8.99 76.98 9.43 77.15 9.87 77.31 10.31 77.46 10.75 77.6 11.19 77.72 11.63 77.83 12.07 77.92 12.51 78 12.95 78.07 13.39 78.12 13.83 78.16 14.27 78.2 14.71 78.22 15.15 78.24 15.59 78.26 16.02 78.27 16.46 78.28 16.9 78.28 17.34 78.29 17.78 78.29 18.22 78.29 18.66 78.29 19.1 78.3 19.54 78.3 19.98 78.3 20.42 78.3 20.86 78.3 21.3 78.3 21.74 78.3 22.18 78.3 22.61 78.3 23.05 78.3 23.49 78.3 23.93 78.3 24.37 78.29 24.81 78.29 25.25 78.29 25.69 78.28 26.13 78.28 26.57 78.27 27.01 78.26 27.45 78.24 27.89 78.21 28.33 78.18 28.77 78.13 29.21 78.08 29.64 78 30.08 77.91 30.52 77.79 30.96 77.65 31.4 77.49 31.84 77.29 32.28 77.06 32.72 76.81 33.16 76.52 33.6 76.22 34.04 75.89 34.48 75.56 34.92 75.22 35.36 74.89 35.8 74.58 36.23 74.3 36.67 74.07 37.11 73.88 37.55 73.75 37.99 73.68 38.43 73.68 38.87 73.74 39.31 73.87 39.75 74.05 40.19 74.29 40.63 74.56 41.07 74.85 41.51 75.17 41.95 75.48 42.39 75.79 42.82 76.07 43.26 76.34 43.7 76.56 44.14 76.75 44.58 76.89 45.02 76.99 45.46 77.05 45.9 77.06 46.34 77.04 46.78 76.97 47.22 76.88 47.66 76.77 48.1 76.64 48.54 76.5 48.98 76.36 49.42 76.22 49.85 76.08 50.29 75.96 50.73 75.86 51.17 75.78 51.61 75.71 52.05 75.65 52.49 75.61 52.93 75.57 53.37 75.53 53.81 75.48 54.25 75.42 54.69 75.33 55.13 75.2 55.57 75.05 56.01 74.85 56.44 74.6 56.88 74.31 57.32 73.98 57.76 73.6 58.2 73.18 58.64 72.73 59.08 72.25 59.52 71.75 59.96 71.22 60.4 70.68 60.84 70.13 61.28 69.57 61.72 69 62.16 68.43 62.6 67.87 63.04 67.3 63.47 66.73 63.91 66.18 64.35 65.62 64.79 65.08 65.23 64.56 65.67 64.05 66.11 63.56 66.55 63.09 66.99 62.65 67.43 62.24 67.87 61.85 68.31 61.5 68.75 61.16 69.19 60.85 69.63 60.55 70.06 60.26 70.5 59.96 70.94 59.64 71.38 59.3 71.82 58.92 72.26 58.49 72.7 58 73.14 57.44 73.58 56.8 74.02 56.07 74.46 55.23 74.9 54.3 75.34 53.24 75.78 52.06 76.22 50.75 76.65 49.29 77.09 47.68 77.53 45.92 77.97 44.01 78.41 41.94 78.85 39.73 79.29 3

p-value SignedRank=0.00000163    p-value Sign=0.00000040


In [3]:
ClassID = readdlm("/home/benavoli/Data/Work_Julia/Tutorial/Julia/Data/ClassifierID.dat", ',')
ClassNames = readdlm("/home/benavoli/Data/Work_Julia/Tutorial/Julia/Data/ClassifierNames.dat", ',')
DatasetID = readdlm("/home/benavoli/Data/Work_Julia/Tutorial/Julia/Data/DatasetID.dat", ',');
DatasetNames = readdlm("/home/benavoli/Data/Work_Julia/Tutorial/Julia/Data/DatasetNames.dat", ',');
Percent_correct = readdlm("/home/benavoli/Data/Work_Julia/Tutorial/Julia/Data/Percent_correct.dat", ',');

cl1b=1 #nbc
cl2b=3 #aode
println("Comparison of ", ClassNames[cl1b,1], " vs. ", ClassNames[cl2b,1])
println()

#Compute mean accuracy
indi=find(x->x==cl1b,ClassID)
indj=find(x->x==cl2b,ClassID)
accNbc=Float64[]
accHnb=Float64[]
for d=1:Int32(maximum(DatasetID))
    indd=find(x->x==d,DatasetID)
    indid=intersect(indi,indd)
    indjd=intersect(indj,indd)
    push!(accNbc,mean(Percent_correct[indid])/100)
    push!(accHnb,mean(Percent_correct[indjd])/100)
end

Comparison of nbc vs. hnb



## NBC-HNB

In [4]:
using HypothesisTests
p1=pvalue(SignedRankTest(accNbc,accHnb))
p2=pvalue(SignTest(accNbc,accHnb))
p=plot_data1(cl1b,cl2b,"all datasets",accNbc-accHnb,-0.15,0.1)
display(p) 
@printf "p-value SignedRank=%0.4f  " p1 
@printf "p-value Sign=%0.4f\n" p2

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 DeltaAcc 
 
 
 -0.6 
 -0.5 
 -0.4 
 -0.3 
 -0.2 
 -0.1 
 0.0 
 0.1 
 0.2 
 0.3 
 0.4 
 0.5 
 -0.40 
 -0.39 
 -0.38 
 -0.37 
 -0.36 
 -0.35 
 -0.34 
 -0.33 
 -0.32 
 -0.31 
 -0.30 
 -0.29 
 -0.28 
 -0.27 
 -0.26 
 -0.25 
 -0.24 
 -0.23 
 -0.22 
 -0.21 
 -0.20 
 -0.19 
 -0.18 
 -0.17 
 -0.16 
 -0.15 
 -0.14 
 -0.13 
 -0.12 
 -0.11 
 -0.10 
 -0.09 
 -0.08 
 -0.07 
 -0.06 
 -0.05 
 -0.04 
 -0.03 
 -0.02 
 -0.01 
 0.00 
 0.01 
 0.02 
 0.03 
 0.04 
 0.05 
 0.06 
 0.07 
 0.08 
 0.09 
 0.10 
 0.11 
 0.12 
 0.13 
 0.14 
 0.15 
 0.16 
 0.17 
 0.18 
 0.19 
 0.20 
 0.21 
 0.22 
 0.23 
 0.24 
 0.25 
 0.26 
 0.27 
 0.28 
 0.29 
 0.30 
 0.31 
 0.32 
 0.33 
 0.34 
 0.35 
 -0.4 
 -0.2 
 0.0 
 0.2 
 0.4 
 -0.40 
 -0.38 
 -0.36 
 -0.34 
 -0.32 
 -0.30 
 -0.28 
 -0.26 
 -0.24 
 -0.22 
 -0.20 
 -0.18 
 -0.16 
 -0.14 
 -0.12 
 -0.10 
 -0.08 
 -0.06 
 -0.04 
 -0.02 
 0.00 
 0.02 
 0.04 
 0.06 
 0.08 
 0.10 
 0.12 
 0.14 
 0.16 
 0.18 
 0.20 
 0.22 
 0.24 
 0.26 
 0.28 
 0.30 
 0.32 
 0.34 
 0.36 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path fill="none" d="M-63.56,78.3 L -62.73 78.29 -61.9 78.29 -61.07 78.29 -60.24 78.29 -59.41 78.28 -58.58 78.28 -57.75 78.27 -56.92 78.25 -56.09 78.23 -55.26 78.2 -54.43 78.16 -53.6 78.1 -52.77 78.02 -51.94 77.93 -51.11 77.8 -50.28 77.65 -49.45 77.46 -48.62 77.23 -47.79 76.97 -46.96 76.66 -46.13 76.33 -45.3 75.96 -44.47 75.56 -43.64 75.16 -42.81 74.75 -41.98 74.35 -41.15 73.98 -40.31 73.66 -39.48 73.39 -38.65 73.2 -37.82 73.08 -36.99 73.05 -36.16 73.1 -35.33 73.24 -34.5 73.46 -33.67 73.74 -32.84 74.08 -32.01 74.45 -31.18 74.85 -30.35 75.26 -29.52 75.67 -28.69 76.05 -27.86 76.42 -27.03 76.75 -26.2 77.04 -25.37 77.29 -24.54 77.51 -23.71 77.69 -22.88 77.84 -22.05 77.95 -21.22 78.05 -20.39 78.12 -19.56 78.17 -18.73 78.21 -17.9 78.24 -17.07 78.26 -16.24 78.27 -15.41 78.28 -14.58 78.29 -13.75 78.29 -12.92 78.29 -12.09 78.29 -11.26 78.3 -10.43 78.3 -9.6 78.3 -8.76 78.3 -7.93 78.3 -7.1 78.3 -6.27 78.3 -5.44 78.3 -4.61 78.3 -3.78 78.3 -2.95 78.3 -2.12 78.3 -1.29 78.3 -0.46 78.3 0.37 78.29 1.2 78.29 2.03 78.29 2.86 78.29 3.69 78.28 4.52 78.27 5.35 78.26 6.18 78.24 7.01 78.21 7.84 78.18 8.67 78.13 9.5 78.08 10.33 78.01 11.16 77.92 11.99 77.81 12.82 77.68 13.65 77.52 14.48 77.35 15.31 77.16 16.14 76.94 16.97 76.72 17.8 76.48 18.63 76.23 19.46 75.99 20.29 75.74 21.12 75.5 21.95 75.27 22.79 75.04 23.62 74.82 24.45 74.6 25.28 74.37 26.11 74.13 26.94 73.88 27.77 73.6 28.6 73.3 29.43 72.96 30.26 72.61 31.09 72.23 31.92 71.85 32.75 71.48 33.58 71.13 34.41 70.82 35.24 70.57 36.07 70.4 36.9 70.33 37.73 70.35 38.56 70.47 39.39 70.7 40.22 71.02 41.05 71.41 41.88 71.87 42.71 72.36 43.54 72.86 44.37 73.35 45.2 73.79 46.03 74.17 46.86 74.46 47.69 74.63 48.52 74.67 49.35 74.56 50.18 74.28 51.01 73.83 51.84 73.2 52.67 72.38 53.5 71.37 54.34 70.17 55.17 68.78 56 67.22 56.83 65.49 57.66 63.62 58.49 61.61 59.32 59.49 60.15 57.29 60.98 55.02 61.81 52.71 62.64 50.39 63.47 48.08 64.3 45.8 65.13 43.58 65.96 41.43 66.79 39.37 67.62 37.4 68.45 35.55 69.28 33.82 70.11 32.21 70.94 30.73 71.77 29.37 72.6 28.14 73.43 27.04 74.26 26.06 75.09 25.19 75.92 24.45 76.75 23.83 77.58 23.31 78.41 22.91 79.24 22.62 80.07 22.45 80.9 22.4 81.73 22.46 82.56 22.66 83.39 23 84.22 23.48 85.05 24.11 85.89 24.9 86.72 25.86 87.55 26.99 88.38 28.29 89.21 29.76 90.04 31.39 90.87 33.18 91.7 35.11 92.53 37.17 93.36 39.35 94.19 41.61 95.02 43.94 95.85 46.32 96.68 48.72 97.51 51.12 98.34 53.

p-value SignedRank=0.0005  p-value Sign=0.0038


In [5]:
ClassID = readdlm("/home/benavoli/Data/Work_Julia/Tutorial/Julia/Data/ClassifierID.dat", ',')
ClassNames = readdlm("/home/benavoli/Data/Work_Julia/Tutorial/Julia/Data/ClassifierNames.dat", ',')
DatasetID = readdlm("/home/benavoli/Data/Work_Julia/Tutorial/Julia/Data/DatasetID.dat", ',');
DatasetNames = readdlm("/home/benavoli/Data/Work_Julia/Tutorial/Julia/Data/DatasetNames.dat", ',');
Percent_correct = readdlm("/home/benavoli/Data/Work_Julia/Tutorial/Julia/Data/Percent_correct.dat", ',');

cl1b=4 #nbc
cl2b=5 #aode
println("Comparison of ", ClassNames[cl1b,1], " vs. ", ClassNames[cl2b,1])
println()

#Compute mean accuracy
indi=find(x->x==cl1b,ClassID)
indj=find(x->x==cl2b,ClassID)
accJ48=Float64[]
accJ48gr=Float64[]
for d=1:Int32(maximum(DatasetID))
    indd=find(x->x==d,DatasetID)
    indid=intersect(indi,indd)
    indjd=intersect(indj,indd)
    push!(accJ48,mean(Percent_correct[indid])/100)
    push!(accJ48gr,mean(Percent_correct[indjd])/100)
end

Comparison of j48 vs. j48gr



## J48 - J48gr (only 30 datasets) 

In [6]:
using HypothesisTests
p1=pvalue(SignedRankTest(accJ48[20:50],accJ48gr[20:50]))
p2=pvalue(SignTest(accJ48[20:50],accJ48gr[20:50]))
p=plot_data1(cl1b,cl2b,"all datasets",accJ48[20:50]-accJ48gr[20:50],-0.013,0.011)
display(p)  
@printf "p-value SignedRank=%0.4f  " p1 
@printf "p-value Sign=%0.4f\n" p2

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 DeltaAcc 
 
 
 -0.07 
 -0.06 
 -0.05 
 -0.04 
 -0.03 
 -0.02 
 -0.01 
 0.00 
 0.01 
 0.02 
 0.03 
 0.04 
 0.05 
 0.06 
 0.07 
 -0.037 
 -0.036 
 -0.035 
 -0.034 
 -0.033 
 -0.032 
 -0.031 
 -0.030 
 -0.029 
 -0.028 
 -0.027 
 -0.026 
 -0.025 
 -0.024 
 -0.023 
 -0.022 
 -0.021 
 -0.020 
 -0.019 
 -0.018 
 -0.017 
 -0.016 
 -0.015 
 -0.014 
 -0.013 
 -0.012 
 -0.011 
 -0.010 
 -0.009 
 -0.008 
 -0.007 
 -0.006 
 -0.005 
 -0.004 
 -0.003 
 -0.002 
 -0.001 
 0.000 
 0.001 
 0.002 
 0.003 
 0.004 
 0.005 
 0.006 
 0.007 
 0.008 
 0.009 
 0.010 
 0.011 
 0.012 
 0.013 
 0.014 
 0.015 
 0.016 
 0.017 
 0.018 
 0.019 
 0.020 
 0.021 
 0.022 
 0.023 
 0.024 
 0.025 
 0.026 
 0.027 
 0.028 
 0.029 
 0.030 
 0.031 
 0.032 
 0.033 
 0.034 
 0.035 
 -0.04 
 -0.02 
 0.00 
 0.02 
 0.04 
 -0.038 
 -0.036 
 -0.034 
 -0.032 
 -0.030 
 -0.028 
 -0.026 
 -0.024 
 -0.022 
 -0.020 
 -0.018 
 -0.016 
 -0.014 
 -0.012 
 -0.010 
 -0.008 
 -0.006 
 -0.004 
 -0.002 
 0.000 
 0.002 
 0.004 
 0.006 
 0.008 
 0.010 
 0.012 
 0.014 
 0.016 
 0.018 
 0.020 
 0.022 
 0.024 
 0.026 
 0.028 
 0.030 
 0.032 
 0.034 
 0.036 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path fill="none" d="M-26.21,78.3 L -25.72 78.29 -25.23 78.29 -24.74 78.29 -24.26 78.29 -23.77 78.29 -23.28 78.29 -22.79 78.28 -22.31 78.27 -21.82 78.27 -21.33 78.25 -20.84 78.24 -20.36 78.22 -19.87 78.2 -19.38 78.17 -18.89 78.13 -18.41 78.08 -17.92 78.03 -17.43 77.96 -16.94 77.88 -16.45 77.78 -15.97 77.67 -15.48 77.54 -14.99 77.4 -14.5 77.24 -14.02 77.06 -13.53 76.87 -13.04 76.67 -12.55 76.45 -12.07 76.23 -11.58 76 -11.09 75.78 -10.6 75.56 -10.12 75.35 -9.63 75.16 -9.14 74.99 -8.65 74.85 -8.17 74.73 -7.68 74.65 -7.19 74.61 -6.7 74.6 -6.22 74.63 -5.73 74.69 -5.24 74.79 -4.75 74.92 -4.27 75.08 -3.78 75.26 -3.29 75.47 -2.8 75.68 -2.32 75.9 -1.83 76.13 -1.34 76.35 -0.85 76.57 -0.37 76.78 0.12 76.98 0.61 77.16 1.1 77.33 1.59 77.48 2.07 77.61 2.56 77.73 3.05 77.83 3.54 77.92 4.02 78 4.51 78.06 5 78.11 5.49 78.15 5.97 78.18 6.46 78.21 6.95 78.23 7.44 78.25 7.92 78.26 8.41 78.27 8.9 78.28 9.39 78.28 9.87 78.29 10.36 78.29 10.85 78.29 11.34 78.29 11.82 78.29 12.31 78.3 12.8 78.3 13.29 78.3 13.77 78.3 14.26 78.3 14.75 78.3 15.24 78.3 15.72 78.29 16.21 78.29 16.7 78.29 17.19 78.29 17.67 78.29 18.16 78.28 18.65 78.28 19.14 78.27 19.62 78.26 20.11 78.24 20.6 78.23 21.09 78.2 21.58 78.17 22.06 78.13 22.55 78.09 23.04 78.03 23.53 77.95 24.01 77.86 24.5 77.75 24.99 77.63 25.48 77.48 25.96 77.3 26.45 77.1 26.94 76.87 27.43 76.61 27.91 76.33 28.4 76.02 28.89 75.68 29.38 75.33 29.86 74.95 30.35 74.56 30.84 74.16 31.33 73.76 31.81 73.37 32.3 72.99 32.79 72.63 33.28 72.29 33.76 72 34.25 71.74 34.74 71.53 35.23 71.38 35.71 71.27 36.2 71.22 36.69 71.23 37.18 71.29 37.66 71.39 38.15 71.54 38.64 71.72 39.13 71.93 39.61 72.16 40.1 72.39 40.59 72.63 41.08 72.86 41.57 73.07 42.05 73.26 42.54 73.43 43.03 73.55 43.52 73.64 44 73.7 44.49 73.71 44.98 73.69 45.47 73.62 45.95 73.53 46.44 73.4 46.93 73.24 47.42 73.06 47.9 72.85 48.39 72.62 48.88 72.37 49.37 72.09 49.85 71.79 50.34 71.47 50.83 71.13 51.32 70.76 51.8 70.38 52.29 69.96 52.78 69.53 53.27 69.08 53.75 68.61 54.24 68.13 54.73 67.65 55.22 67.17 55.7 66.71 56.19 66.26 56.68 65.85 57.17 65.49 57.65 65.17 58.14 64.91 58.63 64.72 59.12 64.6 59.61 64.55 60.09 64.58 60.58 64.68 61.07 64.85 61.56 65.08 62.04 65.36 62.53 65.69 63.02 66.04 63.51 66.4 63.99 66.76 64.48 67.09 64.97 67.39 65.46 67.63 65.94 67.8 66.43 67.88 66.92 67.

p-value SignedRank=0.0822  p-value Sign=0.6636


## J48 - J48gr all datasets (cheating?)

In [7]:
using HypothesisTests
p1=pvalue(SignedRankTest(accJ48,accJ48gr))
p2=pvalue(SignTest(accJ48,accJ48gr))
p=plot_data1(cl1b,cl2b,"all datasets",accJ48-accJ48gr,-0.013,0.011)
display(p)  
@printf "p-value SignedRank=%0.4f  " p1 
@printf "p-value Sign=%0.4f\n" p2

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 DeltaAcc 
 
 
 -0.07 
 -0.06 
 -0.05 
 -0.04 
 -0.03 
 -0.02 
 -0.01 
 0.00 
 0.01 
 0.02 
 0.03 
 0.04 
 0.05 
 0.06 
 0.07 
 -0.037 
 -0.036 
 -0.035 
 -0.034 
 -0.033 
 -0.032 
 -0.031 
 -0.030 
 -0.029 
 -0.028 
 -0.027 
 -0.026 
 -0.025 
 -0.024 
 -0.023 
 -0.022 
 -0.021 
 -0.020 
 -0.019 
 -0.018 
 -0.017 
 -0.016 
 -0.015 
 -0.014 
 -0.013 
 -0.012 
 -0.011 
 -0.010 
 -0.009 
 -0.008 
 -0.007 
 -0.006 
 -0.005 
 -0.004 
 -0.003 
 -0.002 
 -0.001 
 0.000 
 0.001 
 0.002 
 0.003 
 0.004 
 0.005 
 0.006 
 0.007 
 0.008 
 0.009 
 0.010 
 0.011 
 0.012 
 0.013 
 0.014 
 0.015 
 0.016 
 0.017 
 0.018 
 0.019 
 0.020 
 0.021 
 0.022 
 0.023 
 0.024 
 0.025 
 0.026 
 0.027 
 0.028 
 0.029 
 0.030 
 0.031 
 0.032 
 0.033 
 0.034 
 0.035 
 -0.04 
 -0.02 
 0.00 
 0.02 
 0.04 
 -0.038 
 -0.036 
 -0.034 
 -0.032 
 -0.030 
 -0.028 
 -0.026 
 -0.024 
 -0.022 
 -0.020 
 -0.018 
 -0.016 
 -0.014 
 -0.012 
 -0.010 
 -0.008 
 -0.006 
 -0.004 
 -0.002 
 0.000 
 0.002 
 0.004 
 0.006 
 0.008 
 0.010 
 0.012 
 0.014 
 0.016 
 0.018 
 0.020 
 0.022 
 0.024 
 0.026 
 0.028 
 0.030 
 0.032 
 0.034 
 0.036 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path fill="none" d="M-17.42,78.3 L -16.99 78.3 -16.56 78.29 -16.13 78.29 -15.7 78.29 -15.28 78.28 -14.85 78.28 -14.42 78.26 -13.99 78.25 -13.56 78.22 -13.13 78.18 -12.7 78.13 -12.27 78.06 -11.85 77.97 -11.42 77.86 -10.99 77.73 -10.56 77.57 -10.13 77.4 -9.7 77.21 -9.27 77.02 -8.84 76.84 -8.42 76.67 -7.99 76.53 -7.56 76.43 -7.13 76.37 -6.7 76.36 -6.27 76.4 -5.84 76.49 -5.41 76.62 -4.99 76.78 -4.56 76.96 -4.13 77.15 -3.7 77.33 -3.27 77.51 -2.84 77.67 -2.41 77.81 -1.98 77.93 -1.55 78.03 -1.13 78.11 -0.7 78.16 -0.27 78.21 0.16 78.24 0.59 78.26 1.02 78.27 1.45 78.28 1.88 78.29 2.3 78.29 2.73 78.29 3.16 78.3 3.59 78.3 4.02 78.3 4.45 78.3 4.88 78.3 5.31 78.3 5.73 78.3 6.16 78.3 6.59 78.3 7.02 78.3 7.45 78.3 7.88 78.3 8.31 78.3 8.74 78.3 9.17 78.3 9.59 78.3 10.02 78.3 10.45 78.3 10.88 78.3 11.31 78.3 11.74 78.3 12.17 78.3 12.6 78.3 13.02 78.3 13.45 78.3 13.88 78.3 14.31 78.3 14.74 78.3 15.17 78.3 15.6 78.3 16.03 78.3 16.45 78.3 16.88 78.3 17.31 78.3 17.74 78.3 18.17 78.3 18.6 78.3 19.03 78.3 19.46 78.3 19.89 78.3 20.31 78.3 20.74 78.3 21.17 78.3 21.6 78.3 22.03 78.3 22.46 78.3 22.89 78.3 23.32 78.3 23.74 78.3 24.17 78.3 24.6 78.3 25.03 78.29 25.46 78.29 25.89 78.29 26.32 78.28 26.75 78.27 27.17 78.25 27.6 78.22 28.03 78.19 28.46 78.14 28.89 78.07 29.32 77.98 29.75 77.86 30.18 77.72 30.6 77.56 31.03 77.36 31.46 77.14 31.89 76.9 32.32 76.65 32.75 76.39 33.18 76.14 33.61 75.89 34.04 75.68 34.46 75.49 34.89 75.33 35.32 75.22 35.75 75.16 36.18 75.14 36.61 75.17 37.04 75.25 37.47 75.37 37.89 75.54 38.32 75.74 38.75 75.96 39.18 76.21 39.61 76.46 40.04 76.71 40.47 76.96 40.9 77.19 41.32 77.39 41.75 77.55 42.18 77.69 42.61 77.78 43.04 77.83 43.47 77.85 43.9 77.82 44.33 77.76 44.76 77.65 45.18 77.52 45.61 77.36 46.04 77.18 46.47 77 46.9 76.81 47.33 76.63 47.76 76.48 48.19 76.35 48.61 76.26 49.04 76.2 49.47 76.16 49.9 76.14 50.33 76.13 50.76 76.1 51.19 76.05 51.62 75.95 52.04 75.8 52.47 75.59 52.9 75.32 53.33 74.99 53.76 74.62 54.19 74.22 54.62 73.82 55.05 73.42 55.47 73.05 55.9 72.71 56.33 72.4 56.76 72.14 57.19 71.92 57.62 71.72 58.05 71.54 58.48 71.38 58.91 71.23 59.33 71.09 59.76 70.98 60.19 70.91 60.62 70.89 61.05 70.94 61.48 71.07 61.91 71.29 62.34 71.6 62.76 71.97 63.19 72.4 63.62 72.84 64.05 73.26 64.48 73.64 64.91 73.92 65.34 74.09 65.77 74.12 66.19 74.01 66.62 73.75 67.0

p-value SignedRank=0.0006  p-value Sign=0.0095


## Same problems

<div style="float: center;">
  <div align='center' class='content-container'>
    <img src="plots/confused.png" width=370px/>
  </div> 
</div>


## Black and White thinking

* reasoning in the opposite direction (data|hypothesis) and not (hypothesis|data)
* blind decisions $p<\alpha$
  * magnitude of the effect size,
  * the uncertainty,  
  * statistical significance vs. practical significance
* etcetera.

<div style="float: right;">
  <div align='right' class='content-container'>
    <img src="plots/tvb&w.png" width=120px/>
  </div> 
</div>


## A colour thinking
#### Comparison through Bayesian analysis:

<br>



<div style="float: center;">
  <div align='center' class='content-container'>
    <img src="plots/tvcol.png" width=370px/>
  </div> 
</div>